In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("../processed/merged_dataset.csv")
print("Data shape:", data.shape)


Data shape: (374, 51)


In [3]:
SEQ_LEN = 60  # number of time steps per sample
features = data.values

X, y = [], []

# Use the last column as target or define your own target
# For demo, we’ll predict the next timestep’s first feature (you can replace this)
for i in range(len(features) - SEQ_LEN):
    X.append(features[i:i+SEQ_LEN])     # sequence
    y.append(features[i+SEQ_LEN, 0])    # target (e.g., next HR, SpO₂, or label)

X = np.array(X)
y = np.array(y)

print("X shape:", X.shape)
print("y shape:", y.shape)


X shape: (314, 60, 51)
y shape: (314,)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    LSTM(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1)  # or 'sigmoid' if binary classification
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()


c:\Users\Raphael Jay Escosia\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 58, 64)         │         9,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 29, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,993 (175.75 KB)

 Trainable params: 44,993 (175.75 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=32,
    verbose=1
)


Epoch 1/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.3328 - mae: 0.4981 - val_loss: 0.3963 - val_mae: 0.5621
Epoch 2/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.2987 - mae: 0.4703 - val_loss: 0.3885 - val_mae: 0.5552
Epoch 3/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.3001 - mae: 0.4696 - val_loss: 0.3810 - val_mae: 0.5484
Epoch 4/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.3066 - mae: 0.4784 - val_loss: 0.3735 - val_mae: 0.5415
Epoch 5/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2937 - mae: 0.4601 - val_loss: 0.3662 - val_mae: 0.5347
Epoch 6/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.2690 - mae: 0.4334 - val_loss: 0.3589 - val_mae: 0.5281
Epoch 7/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2690 - mae: 0.4395 - val_loss: 0.3518 - val_mae: 0.5216
Epoch 8/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.2421 - mae: 0.4120 - val_loss: 0.3449 - val_mae: 0.5154
Epoch 9/30
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.2855 - mae: 0.4535 - 

In [7]:
loss, mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {mae:.4f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.2216 - mae: 0.3969
Test MAE: 0.3915
